In [56]:
import Module
from scrape_helper import get_module_links, get_courses
from helpers import courses_dict
import sqlite3
import os
from importlib import reload

In [57]:
# Get the list of all courses from the given url
courses = get_courses("https://hpi.de/studium/im-studium/lehrveranstaltungen.html")

# Dictionary to store the courses and their modules (trimmed links)
course_modules = {}

# Dictionary to store trimmed and full module links
module_links_dict = {}

for course in courses:

    # get the list of modules for the given course
    module_links = get_module_links(course)

    # If no modules were found, skip the course
    if module_links == []:
        print("No modules found for course:", course)
        continue

    # Cut the host part of the url
    module_links_trimmed = [link.split("/")[-1] for link in module_links]

    # add the trimmed and full links to the dictionary
    for trimmed, full in zip(module_links_trimmed, module_links):
        module_links_dict[trimmed] = full
    
    # add the course and its modules to the dictionary
    course_modules[course] = module_links_trimmed


Div with class 'tx-ciuniversity' not found for url: https://hpi.de//studium/im-studium/lehrveranstaltungen/professional-skills.html
No modules found for course: https://hpi.de//studium/im-studium/lehrveranstaltungen/professional-skills.html
Div with class 'tx-ciuniversity' not found for url: https://hpi.de/entrepreneurship/home.html
No modules found for course: https://hpi.de/entrepreneurship/home.html


In [127]:
reload(Module)
# create a list of modules of class Module
modules = {}

# create a Module object for each module
for i, (url_trimmed, url) in enumerate(module_links_dict.items()):    

    if i%10 == 0:
        print(f"Fetching data for module {i+1}/{len(module_links_dict)}")

    # create a Module object for the given url
    module = Module.CourseModule(url)
    module.get_landing_page_information()
    module.get_evaluation_metrics()

    # add the module to the list of modules
    modules[url_trimmed] = module

Fetching data for module 1/116
Fetching data for module 11/116
Fetching data for module 21/116
Fetching data for module 31/116
Fetching data for module 41/116
Fetching data for module 51/116
Fetching data for module 61/116
Fetching data for module 71/116
Fetching data for module 81/116
Fetching data for module 91/116
Fetching data for module 101/116
Fetching data for module 111/116


In [128]:
# delete the database if it already exists
try:
    os.remove("hpi_modules.db")
except FileNotFoundError:
    pass
connection = sqlite3.connect("hpi_modules.db")
with open('sqlite_db_setup.sql', 'r') as sql_file:
    sql_queries = sql_file.read()
connection.executescript(sql_queries)
connection.commit()

In [129]:
cursor = connection.cursor()
try:
    cursor.executemany("INSERT INTO courses VALUES (?, ?)", zip(courses_dict.values(), courses_dict.keys()))
    connection.commit()
except sqlite3.IntegrityError:
    print("Courses already exist in the database")
for row in cursor.execute("SELECT * FROM courses"):
    print(row)

('itse_bachelor', 'IT-Systems Engineering BA')
('itse_master', 'IT-Systems Engineering MA')
('dh_master', 'Digital Health MA')
('de_master', 'Data Engineering MA')
('cyber_master', 'Cybersecurity MA')
('sse_master', 'Software Systems Engineering MA')


In [130]:
for module in modules.values():
    try:
        cursor.execute(
            "INSERT INTO modules VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
            (module.url_trimmed,
            module.url,
            module.website,
            module.title,
            module.credits,
            module.evap_grade,
            module.evap_semester,
            module.description)
        )
        connection.commit()
    except sqlite3.IntegrityError:
        print("Module already exists in database: ", module.url_trimmed)

In [131]:
cursor.execute("SELECT COUNT() FROM modules;").fetchall()

[(116,)]

In [132]:
# insert lecturers and their modules into the database
for module in modules.values():
    for lecturer in module.lecturers:
        # Try to insert the lecturer into the database or pass if exists
        cursor.execute("INSERT OR IGNORE INTO lecturers (lecturer_name) VALUES (?)", (lecturer,))
        connection.commit()
        
        # Retrieve the lecturer_id for the current lecturer
        cursor.execute("SELECT lecturer_id FROM lecturers WHERE lecturer_name = ?", (lecturer,))
        lecturer_id = cursor.fetchone()[0] # Assuming lecturer_name is unique
        
        # insert the module and the lecturer id into the module_lecturers table
        try:
            cursor.execute(
                "INSERT INTO module_lecturers (url_trimmed, lecturer_id) VALUES (?, ?)",
                (module.url_trimmed, lecturer_id)
            )
            connection.commit()
        except sqlite3.IntegrityError:
            continue

In [133]:
cursor.execute("SELECT COUNT() FROM modules;").fetchall()

[(116,)]

In [134]:
for module in modules.values():
    for module_group in module.module_groups:
        try:
            cursor.execute("INSERT INTO course_modules VALUES (?, ?, ?, ?)",
                        [
                            module_group[0],    # course abbreviation
                            module.url_trimmed, # module url trimmed
                            module_group[1],    # module group
                            module_group[2]     # submodule group
                        ]) 
            connection.commit()
        except sqlite3.IntegrityError:
            print(f"IntegrityError: {module_group[0]}, {module.url_trimmed}, {module_group[1]}, {module_group[2]}")

IntegrityError: itse_bachelor, wise-23-24-3846-3d_computergrafik-i.html, SB, 
IntegrityError: itse_bachelor, wise-23-24-3846-3d_computergrafik-i.html, SB, 


In [135]:
for row in cursor.execute(
    """
    SELECT * 
    FROM modules 
    JOIN course_modules ON modules.url_trimmed = course_modules.url_trimmed 
    JOIN courses ON courses.course_abbreviation = course_modules.course_abbreviation
    WHERE courses.course_abbreviation LIKE 'dh_master'
    AND modules.evap_grade < 1.2;
    """
    ):
    print(row)

('wise-23-24-3859-applied-data-science-on-real_world-hospital-data-ii.html', 'https://hpi.de/studium/im-studium/lehrveranstaltungen/cybersecurity-ma/lehrveranstaltung/wise-23-24-3859-applied-data-science-on-real_world-hospital-data-ii.html', None, 'Applied data science on real-world hospital data II', 6, 1.1, 'WiSe 2025/26', '<p>Do you want to make a real impact in the field of healthcare? We invite you to seize the opportunity to become an integral part of an innovative healthcare project in collaboration with the Ernst von Bergmann Clinic, Policlinic, and MvZ. Building upon the successful "Hospital Control Center" seminar held in the summer semester of 2023, this project harnesses the knowledge acquired through prior data analysis to enhance hospital planning and, consequently, bolster public health.</p>\n<p>As we\'ve all come to realize during the COVID-19 pandemic, early warnings regarding the emergence of new infection waves are crucial for the well-being of the entire population,

In [115]:
len(modules)

116

In [89]:
cursor.close()
connection.close()